In [1]:
import sys
!{sys.executable} -m pip install librosa

In [2]:
!{sys.executable} -m pip install jupyter_helpers

In [3]:
# Load various imports 
import pandas as pd
import numpy as np
import os
import glob
import wave
import librosa
import librosa.display


In [4]:

def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [5]:
fulldatasetpath = 'F:\Essential Files\Coding Projects\senior design 2\wav_files'

metadata = pd.read_csv('F:\Essential Files\Coding Projects\senior design 2\metadata.csv')

features = []

for row in range(0, 276):
    name = "\\"
    name = name + str(metadata.iloc[row,0])
    file_name = os.path.join(fulldatasetpath + name)
    
    data = extract_features(file_name)
    animal = metadata.iloc[row,1]
    features.append([data, animal])

In [6]:
featuresdf = pd.DataFrame(features, columns=['feature','animal'])

In [7]:
len(featuresdf)

276

In [8]:
!{sys.executable} -m pip install tensorflow

In [9]:
!{sys.executable} -m pip install keras-utils

In [10]:
pip install --ignore-installed --upgrade tensorflow

  Using cached https://files.pythonhosted.org/packages/34/d5/ce8c17971067c0184c9045112b755be5461d5ce5253ef65a367e1298d7c5/tensorflow-2.1.0-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/28/6a/8c1f62c37212d9fc441a7e26736df51ce6f0e38455816445471f10da4f0a/Keras_Preprocessing-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/18/90/b77c328a1304437ab1310b463e533fa7689f4bfc41549593056d812fab8e/tensorflow_estimator-2.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c3/fd/1e86bc4837cc9a3a5faf3db9b1854aa04ad35b5f381f9648fbe81a6f94e4/google_pasta-0.1.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c3/88/97eef84f48fa04fbd6750e62dcceafba6c63c81b7ac1420856c8dcc0a3f9/astor-0.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl
  Using cached https

ERROR: spyder 3.3.6 requires pyqt5<5.13; python_version >= "3", which is not installed.
ERROR: spyder 3.3.6 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
ERROR: astroid 2.2.5 requires typed-ast>=1.3.0; implementation_name == "cpython", which is not installed.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\Axel\\Anaconda3\\Lib\\site-packages\\numpy\\.libs\\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



In [11]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

Using TensorFlow backend.


In [12]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.animal.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [13]:
num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [14]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [15]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                

In [16]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='F:\Essential Files\Coding Projects\senior design 2\weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 220 samples, validate on 56 samples
Epoch 1/100
220/220 [==============================] - 0s 467us/step - loss: 19.2940 - accuracy: 0.4909 - val_loss: 4.4188 - val_accuracy: 0.6607

Epoch 00001: val_loss improved from inf to 4.41877, saving model to F:\Essential Files\Coding Projects\senior design 2\weights.best.basic_mlp.hdf5
Epoch 2/100
220/220 [==============================] - 0s 199us/step - loss: 17.9792 - accuracy: 0.4273 - val_loss: 4.3092 - val_accuracy: 0.6607

Epoch 00002: val_loss improved from 4.41877 to 4.30920, saving model to F:\Essential Files\Coding Projects\senior design 2\weights.best.basic_mlp.hdf5
Epoch 3/100
220/220 [==============================] - 0s 77us/step - loss: 15.9687 - accuracy: 0.4500 - val_loss: 7.8025 - val_accuracy: 0.6607

Epoch 00003: val_loss did not improve from 4.30920
Epoch 4/100
220/220 [==============================] - 0s 82us/step - loss: 11.2619 - accuracy: 0.5909 - val_loss: 4.0040 - val_accuracy: 0.7857

Epoch 00004: val_los

In [17]:

# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.949999988079071
Testing Accuracy:  0.9107142686843872


In [18]:
def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        #print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])

In [19]:
def print_prediction(file_name):
    prediction_feature = extract_feature(file_name) 

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector)
    print("The predicted class is:", predicted_class[0], '\n') 
    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [20]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\kitty_1.wav' 
print_prediction(filename)

The predicted class is: cat 

cat 		 :  0.79212391376495361328125000000000
chicken 		 :  0.03475961089134216308593750000000
dog 		 :  0.17311644554138183593750000000000


In [21]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\doggo_1.wav' 
print_prediction(filename)

The predicted class is: chicken 

cat 		 :  0.23848669230937957763671875000000
chicken 		 :  0.56453317403793334960937500000000
dog 		 :  0.19698019325733184814453125000000


In [22]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\kitty_2.wav' 
print_prediction(filename)

The predicted class is: cat 

cat 		 :  0.68192803859710693359375000000000
chicken 		 :  0.07389143854379653930664062500000
dog 		 :  0.24418056011199951171875000000000


In [23]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\doggo_2.wav' 
print_prediction(filename)

The predicted class is: chicken 

cat 		 :  0.31097909808158874511718750000000
chicken 		 :  0.35476472973823547363281250000000
dog 		 :  0.33425620198249816894531250000000


In [38]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\kitty_3.wav' 
print_prediction(filename)

The predicted class is: cat 

cat 		 :  0.61128163337707519531250000000000
dog 		 :  0.38871833682060241699218750000000


In [39]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\doggo_3.wav' 
print_prediction(filename)

The predicted class is: dog 

cat 		 :  0.30033823847770690917968750000000
dog 		 :  0.69966185092926025390625000000000


In [31]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\dog_barking_20.wav' 
print_prediction(filename)

The predicted class is: dog 

cat 		 :  0.01554717961698770523071289062500
dog 		 :  0.98445278406143188476562500000000


In [ ]:
filename = 'F:\Essential Files\Coding Projects\senior design 2\wav_files\dog_barking_100.wav' 
print_prediction(filename)